In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
pip install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=8f9082d46856838499aa91b7fb6a920fc3a5b97d1fb1f73c5d4945b13a12e389
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
"""
run:
python -m generate_data generate_instructions_from_websites
"""
import os
import re
import time

import openai
from openai import openai_object
import pandas as pd
from pathlib import Path
import json
import requests
from bs4 import BeautifulSoup
from requests.exceptions import Timeout
from typing import Union
import fire


'\ndef format_data() -> None:\n    txt = Path(\'Q&A_raw.txt\').read_text()\n    txt = re.sub(r"\x08Q:\\s", "*Q*: ", txt)\n    txt.replace(\'\n\', \'\')\n    txt = txt.split("*Q*: ")[1:]\n    qlst = []\n    alst = []\n    for pair in txt:\n        if "A: " in pair:\n            ind = pair.index("A: ")\n            q = pair[:ind]\n            a = pair[ind + 2:]\n            qlst.append(q.strip())\n            alst.append(a.strip())\n        else :\n            continue\n\n    qa_dict = {"instruction":qlst, "output":alst}\n    df = pd.DataFrame(qa_dict)\n    df["input"] = ""\n    df = df[["instruction", "input", "output"]]\n\n    df_json = pd.read_json(\'alpaca_data_cleaned.json\')\n    df_json = df_json.sample(frac=0.5)\n\n    df_qa = pd.read_csv("Q&A_raw.csv")\n    df_qa["instruction"] = ""\n    df_qa["input"] = [word[2:] for word in df_qa["Questions"]]\n    df_qa["output"] = [word[2:] for word in df_qa["Answers"]]\n    df_qa.drop(["Questions", "Answers"],axis=1, inplace=True)\n\n    df

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-sk-sk"
#"sk-sk"
openai.api_key = os.getenv("OPENAI_API_KEY")

StrOrOpenAIObject = Union[str, openai_object.OpenAIObject]

In [ ]:
def get_text_from_url(url: str) -> list[list[str]]:
    print("inside the function: get_text_from_url")
    try:
        response = requests.get(url, timeout=10)
    except requests.exceptions.Timeout:
        print(f"Skipping {url} due to timeout")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    text = [p.text for p in soup.find_all('p') if len(p.text) > 50]

    tokens = ' '.join(text).split()

    # split page up into x token chunks
    truncated_text = split_tokens(tokens, 850)

    return truncated_text

In [ ]:
def split_tokens(token_list: list[str], max_tokens: int = 1000) -> list[list[str]]:
    return [token_list[i:i+max_tokens] for i in range(0, len(token_list), max_tokens)]


In [ ]:
def encode_prompt(website: str) -> list[str]:
    prompt = open('./prompt.txt', 'r').read()
    prompts = []
    website_chunks = get_text_from_url(website)
    for website_text in website_chunks:
        prompts.append(prompt.format(website_text))
    return prompts

In [ ]:
def get_response(
    website: str,
    temperature: float = 0.6,
    max_tokens: int = 500,
    top_p: float = 1,
    n: int = 1,
    steam: bool = False,
    frequency_penalty: float = 0.1,
    presence_penalty: float = 0,
    logit_bias={"50256": -100},  # prevent the  token from being generated
) -> list[StrOrOpenAIObject]:

    prompts = encode_prompt(website)
    if not prompts:
        return []
    start_time = time.time()  # measure the start time of the response

    responses = []
    rpmCount: int = 0
    for prompt in prompts:
        print("NEXT STEP is: openai.ChatCompletion.create")
        rpmCount = rpmCount+ 1
        print("request count is: ",rpmCount)
        if  rpmCount%2 ==0:
            print("Yes its the 2nd pattern RPM")
            time.sleep(60)
            print("Scheduled sleep of 60 secs done")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {'role': 'user', 'content': prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            n=n,
            stream=steam,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            logit_bias=logit_bias
        )
        responses.append(response)

    end_time = time.time()  # measure the end time of the response

    # print the time it took for the response for a website
    print(f" Response Time: {end_time - start_time:.2f} seconds")

    return responses

In [ ]:
def post_process_response(response) -> str:
    if not response:
        return ""

    raw_instructions = response["choices"][0]["message"]["content"]
    QA_pairs = raw_instructions.split("\n\n")

    # remove the last QA pair if it's incomplete
    if response["choices"][0]["finish_reason"] == "length":
        QA_pairs.pop()

    return '\n\n'.join(QA_pairs)

In [ ]:
def add_to_df(input: str, df: pd.DataFrame) -> pd.DataFrame:
    if not input:
        return df

    qna_list = input.split('\n\n')
    new_df = pd.DataFrame(
        [qna.split('\n') for qna in qna_list if len(qna.split('\n')) == 2], columns=['Questions', 'Answers']
    )

    result_df = pd.concat([df, new_df], ignore_index=True)

    return result_df

In [ ]:
def get_raw_data() -> None:
    with open("./sources.txt", "r", encoding="utf-8-sig") as f:
        websites = [line.strip() for line in f.readlines()]

    QA_df = pd.DataFrame(columns=['Questions', 'Answers'])
    for website in websites:
        print("website link is: ",website)
        responses = get_response(website)
        if not responses:
            continue
        for response in responses:
            response_text = post_process_response(response)
            QA_df = add_to_df(response_text, QA_df)
        print(f"Total QA pairs: {QA_df.shape[0]}")
    QA_df.to_csv("Q&A_raw.csv", index=False)

In [ ]:
def generate_instructions_from_websites() -> None:
    get_raw_data()
   #format_data()

"""
if __name__ == "__main__":
    fire.Fire(globals())
"""

'\nif __name__ == "__main__":\n    fire.Fire(globals())\n'

In [25]:
generate_instructions_from_websites()

website link is:  https://www.investopedia.com/personal-finance/most-common-financial-mistakes/
inside the function: get_text_from_url
NEXT STEP is: openai.ChatCompletion.create
request count is:  1
NEXT STEP is: openai.ChatCompletion.create
request count is:  2
Yes its the 2nd pattern RPM
Scheduled sleep of 60 secs done
 Response Time: 91.24 seconds
Total QA pairs: 8
website link is:  https://www.investopedia.com/articles/pf/09/financial-responsibility.asp
inside the function: get_text_from_url
NEXT STEP is: openai.ChatCompletion.create
request count is:  1
NEXT STEP is: openai.ChatCompletion.create
request count is:  2
Yes its the 2nd pattern RPM
Scheduled sleep of 60 secs done
 Response Time: 88.65 seconds
Total QA pairs: 18
website link is:  https://www.investopedia.com/personal-finance-calendar-5092591
inside the function: get_text_from_url
NEXT STEP is: openai.ChatCompletion.create
request count is:  1
NEXT STEP is: openai.ChatCompletion.create
request count is:  2
Yes its the 2n